In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Conv2DTranspose, Reshape, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

In [ ]:
# Load dataset
dataset_dir = "dataset"

In [ ]:
# Get image size from the first image
first_image_file = os.listdir(f"{dataset_dir}/Non Demented")[0]
img = plt.imread(f"{dataset_dir}/Non Demented/{first_image_file}")

img_height, img_width, _ = img.shape

print(f"Image size: {img_height}x{img_width}")

In [ ]:
# Set batch size
batch_size = 16

In [ ]:
# Using Tensor image generator to load train-test generators
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.1
)

train_generator = datagen.flow_from_directory(dataset_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode="categorical", subset="training", shuffle=True, seed=42)
test_generator = datagen.flow_from_directory(dataset_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode="categorical", subset="validation", shuffle=True, seed=42)

In [ ]:
# def crop_image(image):
#     """Crops the black borders around an image."""
#     # Convert to grayscale
#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Threshold the image to get the binary mask
#     _, thresholded_image = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

#     # Find contours
#     contours, _ = cv2.findContours(thresholded_image.astype("uint8"), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     # If contours were found, crop the image
#     if contours:
#         x, y, width, height = cv2.boundingRect(contours[0])
#         cropped_image = image[y : y + height, x : x + width]
#         return cropped_image

#     # If no contours were found, return the original image
#     return image


# def get_cropped_images(generator, batch_size):
#     """Returns a new ImageDataGenerator with cropped images and their corresponding labels from a generator."""
#     cropped_images = []
#     labels = []

#     # Iterate over all batches in the generator
#     for i in range(len(generator)):
#         images_batch, labels_batch = generator[i]

#         # Iterate over all images in the batch
#         for image, label in zip(images_batch, labels_batch):
#             cropped_image = crop_image(image)
#             cropped_images.append(cropped_image)
#             labels.append(label)

#     cropped_images = np.array(cropped_images)
#     labels = np.array(labels)

#     # Create a new ImageDataGenerator with cropped images and labels
#     cropped_generator = ImageDataGenerator()
#     cropped_generator.fit(cropped_images)

#     return cropped_generator.flow(cropped_images, labels, batch_size=batch_size, shuffle=True)


# # Get the cropped images and labels
# train_generator = get_cropped_images(train_generator, batch_size)
# test_generator = get_cropped_images(test_generator, batch_size)

In [ ]:
# # Show the cropped images
# fig, axes = plt.subplots(1, 2, figsize=(10, 5))
# axes[0].imshow(X_train[10])
# axes[0].set_title("Cropped Image")
# axes[1].imshow(train_generator[0][0][10])
# axes[1].set_title("Original Image")
# plt.show()

In [ ]:
# Get class labels
class_labels = list(train_generator.class_indices.keys())

# Display sample images
plt.figure(figsize=(8, 8))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    for X_batch, Y_batch in train_generator:
        image = X_batch[0]
        label = Y_batch[0]
        plt.imshow(image)
        plt.title(class_labels[np.argmax(label)])
        break
plt.tight_layout()
plt.show()

In [ ]:
# Number of classes
num_classes = len(train_generator.class_indices)

In [ ]:
# Set number of epochs
epochs_number = 200

#### Score metrics

In [ ]:
# def recall_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall


# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision


# def specificity_m(y_true, y_pred):
#     true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
#     possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
#     specificity = true_negatives / (possible_negatives + K.epsilon())
#     return specificity


# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [ ]:
# Compile model
adam_optimizer = Adam(learning_rate=0.001)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

#### Testing pretrained ResNet-50 model


In [ ]:
# Load the ResNet50 model
base_resNet_model = ResNet50V2(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# Add custom classification head
x = base_resNet_model.output
x = AveragePooling2D(pool_size=(4, 4))(x)
x = Flatten(name="flatten")(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.25)(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Create model
resnet_model = Model(inputs=base_resNet_model.input, outputs=predictions)

# Freeze convolutional layers
for layer in base_resNet_model.layers:
    layer.trainable = False

resnet_model.compile(optimizer=adam_optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
# resnet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", f1_m])

In [ ]:
# Train model
resNet_history = resnet_model.fit(
    train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs_number, validation_data=test_generator, validation_steps=test_generator.samples // batch_size, callbacks=[early_stopping]
)

In [ ]:
# Plot training and validation accuracy
plt.plot(resNet_history.history["accuracy"], label="Training accuracy")
plt.plot(resNet_history.history["val_accuracy"], label="Validation accuracy")
plt.title("Training and validation accuracy - ResNet50V2")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# Save trained model
resnet_model.save("resNet_model.h5")

# Save history
with open("resNet_history.pkl", "wb") as f:
    pkl.dump(resNet_history.history, f)

#### Testing DenseNet-121 model


In [ ]:
# Load pre-trained DenseNet121 model
base_denseNet_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# Add custom classification head
x = base_denseNet_model.output
x = AveragePooling2D(pool_size = (4,4))(x)
x = Flatten(name= 'flatten')(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.25)(x)
predictions = Dense(num_classes, activation = 'softmax')(x)

# Create model
denseNet_model = Model(inputs=base_denseNet_model.input, outputs=predictions)

# Freeze convolutional layers
for layer in base_denseNet_model.layers:
    layer.trainable = False

# Compile model
denseNet_model.compile(optimizer=adam_optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
# denseNet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", f1_m])

In [ ]:
# Train model
denseNet_history = denseNet_model.fit(
    train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs_number, validation_data=test_generator, validation_steps=test_generator.samples // batch_size, callbacks=[early_stopping]
)

In [ ]:
plt.plot(denseNet_history.history["accuracy"], label="Training accuracy")
plt.plot(denseNet_history.history["val_accuracy"], label="Validation accuracy")
plt.title("Training and validation accuracy - DenseNet121")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# Save trained model
denseNet_model.save("denseNet_model.h5")

# Save history
with open("denseNet_history.pkl", "wb") as f:
    pkl.dump(denseNet_history.history, f)

#### Testing EfficientNetB7 model


In [ ]:
# Load the ResNet50 model
base_efficientNet_model = EfficientNetB7(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# Add custom classification head
x = base_efficientNet_model.output
x = AveragePooling2D(pool_size = (4,4))(x)
x = Flatten(name= 'flatten')(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.25)(x)
predictions = Dense(num_classes, activation = 'softmax')(x)

# Create model
efficientNet_model = Model(inputs=base_efficientNet_model.input, outputs=predictions)

# Freeze convolutional layers
for layer in base_efficientNet_model.layers:
    layer.trainable = False

# Compile model
efficientNet_model.compile(optimizer=adam_optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
# efficientNet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", f1_m])

In [ ]:
# Train the model
efficientNet_history = efficientNet_model.fit(
    train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs_number, validation_data=test_generator, validation_steps=test_generator.samples // batch_size, callbacks=[early_stopping]
)

In [ ]:
# Save the trained model
efficientNet_model.save("efficientNet_model.h5")

# Save the history
with open("efficientNet_history.pkl", "wb") as f:
    pkl.dump(efficientNet_history.history, f)

In [ ]:
plt.plot(efficientNet_history.history["accuracy"], label="Training accuracy")
plt.plot(efficientNet_history.history["val_accuracy"], label="Validation accuracy")
plt.title("Training and validation accuracy - EfficientNetB7")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

#### Testing custom CNN model


In [ ]:
# # Custom CNN architecture
# custom_model = Sequential(
#     [
#         Conv2D(32, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)),
#         MaxPooling2D((2, 2)),
#         Conv2D(64, (3, 3), activation="relu"),
#         MaxPooling2D((2, 2)),
#         Conv2D(128, (3, 3), activation="relu"),
#         MaxPooling2D((2, 2)),
#         Conv2D(128, (3, 3), activation="relu"),
#         MaxPooling2D((2, 2)),
#         Flatten(),
#         Dense(512, activation="relu"),
#         Dropout(0.5),
#         Dense(num_classes, activation="softmax"),
#     ]
# )

# # Compile the model
# custom_model.compile(optimizer=adam_optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
# # custom_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", f1_m])

In [ ]:
# # Train the model
# custom_CNN_history = custom_model.fit(
#     train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs_number, validation_data=test_generator, validation_steps=test_generator.samples // batch_size, callbacks=[early_stopping]
# )

In [ ]:
# # Save the trained model
# custom_model.save("custom_CNN_model.h5")

In [ ]:
# plt.plot(custom_CNN_history.history["accuracy"], label="Training accuracy")
# plt.plot(custom_CNN_history.history["val_accuracy"], label="Validation accuracy")
# plt.title("Training and validation accuracy - Custom CNN")
# plt.xlabel("Epoch")
# plt.ylabel("Accuracy")
# plt.legend()
# plt.show()

#### Evaluate models


In [ ]:
# Table comparing the performance of models
resNet_accuracy = resNet_history.history["accuracy"][-1]
resNet_val_accuracy = resNet_history.history["val_accuracy"][-1]
denseNet_accuracy = denseNet_history.history["accuracy"][-1]
denseNet_val_accuracy = denseNet_history.history["val_accuracy"][-1]
efficientNet_accuracy = efficientNet_history.history["accuracy"][-1]
efficientNet_val_accuracy = efficientNet_history.history["val_accuracy"][-1]
# custom_CNN_accuracy = custom_CNN_history.history["accuracy"][-1]
# custom_CNN_val_accuracy = custom_CNN_history.history["val_accuracy"][-1]

# resNet_f1 = resNet_history.history["f1_m"][-1]
# resNet_val_f1 = resNet_history.history["val_f1_m"][-1]
# denseNet_f1 = denseNet_history.history["f1_m"][-1]
# denseNet_val_f1 = denseNet_history.history["val_f1_m"][-1]
# efficientNet_f1 = efficientNet_history.history["f1_m"][-1]
# efficientNet_val_f1 = efficientNet_history.history["val_f1_m"][-1]
# custom_CNN_f1 = custom_CNN_history.history["f1_m"][-1]
# custom_CNN_val_f1 = custom_CNN_history.history["val_f1_m"][-1]

model_comparison = pd.DataFrame(
    {
        "Model": ["ResNet50V2", "DenseNet121", "Custom CNN", "EfficientNetB7"],
        "Train Accuracy": [resNet_accuracy, denseNet_accuracy, efficientNet_accuracy],
        "Validation Accuracy": [resNet_val_accuracy, denseNet_val_accuracy, efficientNet_val_accuracy],
        # "Training F1 Score": [resNet_f1, denseNet_f1, custom_CNN_f1, efficientNet_f1],
        # "Validation F1 Score": [resNet_val_f1, denseNet_val_f1, custom_CNN_val_f1, efficientNet_val_f1],
    }
)

model_comparison